In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

import numpy as np
import matplotlib.pyplot as plt
from simulation import simulate_emission_quadratic, simulate_log_damage

np.random.seed(0)

In [ ]:
# Model parameters
δ = 0.01
η = 0.032
median = 1.75/1000
h_hat = 0.2
σ_n = 1.5
γ_low = 0.012
γ_high = 0.024

ξ = σ_n/h_hat*δ*η

## 1. Emission trajectory

In [ ]:
# Simulate emission trajectory
r_start = 6000
T = 400
args_trace_ϕ = (-20, -1, 1000, 1e-9, 1e-3)

Et_low, r_grid_low, _, e_grid_low  = simulate_emission_quadratic(δ, η, median*γ_low, ξ, σ_n,
                                     args_trace_ϕ=args_trace_ϕ,
                                     r_start=r_start, T=T)
Et_high, r_grid_high, _, e_grid_high = simulate_emission_quadratic(δ, η, median*γ_high, ξ, σ_n,
                                      args_trace_ϕ=args_trace_ϕ,
                                      r_start=r_start, T=T)

In [ ]:
plt.plot(Et_low, label='γ = 0.012')
plt.plot(Et_high, label='γ = 0.024')
plt.xlabel('Year')
plt.ylabel('Emission')
plt.title('Figure 1: Emission Trajectory')
plt.ylim(0, 20)
plt.legend()
plt.savefig('figure_1.pdf')
plt.show()

In [ ]:
plt.plot(r_grid_low, e_grid_low, label='γ = 0.012')
plt.plot(r_grid_high, e_grid_high, label='γ = 0.024')
plt.legend()
plt.xlabel('Reserve')
plt.xlim(-400, 6000)
plt.title('Figure 2: Emission as a function of reserve')
plt.savefig('figure_2.pdf')
plt.show()

## 2. Implied $\hat{h}$

In [ ]:
E_tilde_low = δ*η/(median*γ_low)
E_tilde_high = δ*η/(median*γ_high)
h_hat_implied_low = Et_low/E_tilde_low * h_hat
h_hat_implied_high = Et_high/E_tilde_high * h_hat

In [ ]:
plt.plot(h_hat_implied_low, label='γ = 0.012')
plt.plot(h_hat_implied_high, label='γ = 0.024')
plt.xlabel('Year')
plt.ylabel(r'$\hat{h}$')
plt.title(r'Implied $\hat{h}$')
plt.legend()
plt.show()

## 3. Damage and temperature change

In [ ]:
T = 100
# Simulate damages
exp_avg_response_low = median * γ_low / (1-η)
exp_avg_response_high = median * γ_high / (1-η)
Ws = np.random.normal(0., 1., (100_000, T))
log_damage_low = simulate_log_damage(exp_avg_response_low, σ_n, Et_low, Ws)
log_damage_high = simulate_log_damage(exp_avg_response_high, σ_n, Et_high, Ws)
proportional_damage_low = np.exp(-log_damage_low)
proportional_damage_high = np.exp(-log_damage_high)

In [ ]:
# Histogram for temperature change divided by cumulative emissions
plt.hist(log_damage_low[:, T-1] / Et_low[:T].sum() / γ_low, bins=200)
plt.title('Temperature change divided by cumulative emissions, γ = 0.012')
plt.show()
# Histogram for temperature change
plt.hist(log_damage_low[:, T-1] / γ_low, bins=200)
plt.title('Temperature change, γ = 0.012')
plt.show()

In [ ]:
# Histogram for temperature change divided by cumulative emissions
plt.hist(log_damage_high[:, T-1] / Et_high[:T].sum() / γ_high, bins=200)
plt.title('Temperature change divided by cumulative emissions, γ = 0.024')
plt.show()
# Histogram for temperature change
plt.hist(log_damage_high[:, T-1] / γ_high, bins=200)
plt.title('Temperature change, γ = 0.024')
plt.show()

## 4. Compare $\xi$

In [ ]:
# Simulate emission trajectory, current ξ
args_trace_ϕ = (-20, -1, 1000, 1e-9, 1e-3)
r_start = 6000
T = 500
Et_low, _, _, _ = simulate_emission_quadratic(δ, η, median*γ_low, ξ, σ_n,
                                     args_trace_ϕ=args_trace_ϕ,
                                     r_start=r_start, T=T)
Et_high, _, _, _  = simulate_emission_quadratic(δ, η, median*γ_high, ξ, σ_n,
                                      args_trace_ϕ=args_trace_ϕ,
                                      r_start=r_start, T=T)

# Simulate emission trajectory, large ξ
args_trace_ϕ = (-20, -9, 1000, 1e-9, 1e-3)
Et_low_large_ξ, _, _, _  = simulate_emission_quadratic(δ, η, median*γ_low, ξ*100_000, σ_n,
                                             args_trace_ϕ=args_trace_ϕ,
                                             r_start=r_start, T=T)
args_trace_ϕ = (-20, -7, 1000, 1e-9, 1e-3)
Et_high_large_ξ, _, _, _   = simulate_emission_quadratic(δ, η, median*γ_high, ξ*100_000, σ_n,
                                               args_trace_ϕ=args_trace_ϕ,
                                               r_start=r_start, T=T)

In [ ]:
plt.plot(Et_low, label='γ = 0.012')
plt.plot(Et_low_large_ξ, label='γ = 0.024')
plt.xlabel('Year')
plt.ylabel('Emission')
plt.title('Emission Trajectory')
plt.ylim(0, 20)
plt.legend()
plt.show()

In [ ]:
plt.plot(Et_high, label='γ = 0.012')
plt.plot(Et_high_large_ξ, label='γ = 0.024')
plt.xlabel('Year')
plt.ylabel('Emission')
plt.title('Emission Trajectory')
plt.ylim(0, 20)
plt.legend()
plt.show()

## More experiments on $\xi$

In [ ]:
r_start = 6000
T = 500

# Simulate emission trajectory, 0.01x ξ
args_trace_ϕ = (-20, 0, 1000, 1e-9, 1e-3)
Et_low_001x, r_grid_low_001x, _, e_grid_low_001x  = simulate_emission_quadratic(δ, η, median*γ_low, ξ*0.01, σ_n,
                                                         args_trace_ϕ=args_trace_ϕ,
                                                         r_start=r_start, T=T)
args_trace_ϕ = (-20, 0, 1000, 1e-9, 1e-3)
Et_high_001x, r_grid_high_001x, _, e_grid_high_001x = simulate_emission_quadratic(δ, η, median*γ_high, ξ*0.01, σ_n,
                                                           args_trace_ϕ=args_trace_ϕ,
                                                           r_start=r_start, T=T)

# Simulate emission trajectory, 0.05x ξ
args_trace_ϕ = (-20, 0, 1000, 1e-9, 1e-3)
Et_low_005x, r_grid_low_005x, _, e_grid_low_005x  = simulate_emission_quadratic(δ, η, median*γ_low, ξ*0.05, σ_n,
                                                         args_trace_ϕ=args_trace_ϕ,
                                                         r_start=r_start, T=T)
args_trace_ϕ = (-20, 0, 1000, 1e-9, 1e-3)
Et_high_005x, r_grid_high_005x, _, e_grid_high_005x = simulate_emission_quadratic(δ, η, median*γ_high, ξ*0.05, σ_n,
                                                           args_trace_ϕ=args_trace_ϕ,
                                                           r_start=r_start, T=T)

# Simulate emission trajectory, 0.1x ξ
args_trace_ϕ = (-20, 0, 1000, 1e-9, 1e-3)
Et_low_01x, r_grid_low_01x, _, e_grid_low_01x = simulate_emission_quadratic(δ, η, median*γ_low, ξ*0.1, σ_n,
                                     args_trace_ϕ=args_trace_ϕ,
                                     r_start=r_start, T=T)
Et_high_01x, r_grid_high_01x, _, e_grid_high_01x  = simulate_emission_quadratic(δ, η, median*γ_high, ξ*0.1, σ_n,
                                      args_trace_ϕ=args_trace_ϕ,
                                      r_start=r_start, T=T)

# Simulate emission trajectory, 1x ξ
args_trace_ϕ = (-20, -1, 1000, 1e-9, 1e-3)
Et_low, r_grid_low, _, e_grid_low = simulate_emission_quadratic(δ, η, median*γ_low, ξ, σ_n,
                                     args_trace_ϕ=args_trace_ϕ,
                                     r_start=r_start, T=T)
Et_high, r_grid_high, _, e_grid_high  = simulate_emission_quadratic(δ, η, median*γ_high, ξ, σ_n,
                                      args_trace_ϕ=args_trace_ϕ,
                                      r_start=r_start, T=T)

In [ ]:
E_tilde_low = δ*η/(median*γ_low)
E_tilde_high = δ*η/(median*γ_high)

h_hat_implied_low_001x = Et_low_001x/E_tilde_low * h_hat
h_hat_implied_low_005x = Et_low_005x/E_tilde_low * h_hat
h_hat_implied_low_01x = Et_low_01x/E_tilde_low * h_hat
h_hat_implied_low = Et_low/E_tilde_low * h_hat

h_hat_implied_high_001x = Et_high_001x/E_tilde_high * h_hat
h_hat_implied_high_005x = Et_high_005x/E_tilde_high * h_hat
h_hat_implied_high_01x = Et_high_01x/E_tilde_high * h_hat
h_hat_implied_high = Et_high/E_tilde_high * h_hat

In [ ]:
plt.plot(r_grid_low_001x, e_grid_low_001x, label='0.01x ξ')
plt.plot(r_grid_low_005x, e_grid_low_005x, label='0.05x ξ')
plt.plot(r_grid_low_01x, e_grid_low_01x, label='0.1x ξ')
plt.plot(r_grid_low, e_grid_low, label='1x ξ')
plt.legend()
plt.xlabel('Reserve')
plt.title('Figure 1: Emission as a function of reserve (low damage)')
plt.savefig('figure_1.pdf')
plt.show()

In [ ]:
plt.plot(Et_low_001x, label='0.01x ξ')
plt.plot(Et_low_005x, label='0.05x ξ')
plt.plot(Et_low_01x, label='0.1x ξ')
plt.plot(Et_low, label='1x ξ')
plt.legend()
plt.xlabel('Year')
plt.ylim(0,20)
plt.title('Figure 2: Emission trajectory (low damage)')
plt.savefig('figure_2.pdf')
plt.show()

In [ ]:
plt.plot(h_hat_implied_low_001x, label='0.01x ξ')
plt.plot(h_hat_implied_low_005x, label='0.05x ξ')
plt.plot(h_hat_implied_low_01x, label='0.1x ξ')
plt.plot(h_hat_implied_low, label='1x ξ')
plt.legend()
plt.xlabel('Year')
plt.ylim(0,0.3)
plt.title(r'Figure 3: Implied $\hat{h}$ (low damage)')
plt.savefig('figure_3.pdf')
plt.show()

In [ ]:
plt.plot(r_grid_high_001x, e_grid_high_001x, label='0.01x ξ')
plt.plot(r_grid_high_005x, e_grid_high_005x, label='0.05x ξ')
plt.plot(r_grid_high_01x, e_grid_high_01x, label='0.1x ξ')
plt.plot(r_grid_high, e_grid_high, label='1x ξ')
plt.legend()
plt.xlabel('Reserve')
plt.title('Figure 4: Emission as a function of reserve (high damage)')
plt.savefig('figure_4.pdf')
plt.show()

In [ ]:
plt.plot(Et_high_001x, label='0.01x ξ')
plt.plot(Et_high_005x, label='0.05x ξ')
plt.plot(Et_high_01x, label='0.1x ξ')
plt.plot(Et_high, label='1x ξ')
plt.legend()
plt.xlabel('Year')
plt.ylim(0,5)
plt.title('Figure 5: Emission trajectory (high damage)')
plt.savefig('figure_5.pdf')
plt.show()

In [ ]:
plt.plot(h_hat_implied_high_001x, label='0.01x ξ')
plt.plot(h_hat_implied_high_005x, label='0.05x ξ')
plt.plot(h_hat_implied_high_01x, label='0.1x ξ')
plt.plot(h_hat_implied_high, label='1x ξ')
plt.legend()
plt.xlabel('Year')
plt.ylim(0,0.3)
plt.title(r'Figure 6: Implied $\hat{h}$ (high damage)')
plt.savefig('figure_6.pdf')
plt.show()